In [0]:
import pyspark
from pyspark.sql import DataFrameWriter
import pandas as pd
from pyspark.sql.functions import *

In [0]:
spark.conf.set("fs.azure.account.key.carehealthcareblobsa.dfs.core.windows.net",
    dbutils.secrets.get(scope="healthcarescope", key="blobaccesskey"))

In [0]:
patient_data = spark.read.csv("abfss://rawhealthcaredata@carehealthcareblobsa.dfs.core.windows.net/Patient_records.csv", header=True, inferSchema=True)

In [0]:
#patient_data.show(10,False)

+----------+------------+--------------+------------------+--------------+----------------+------------+-----------+
|Patient_id|Patient_name|patient_gender|patient_birth_date|patient_phone |disease_name    |city        |hospital_id|
+----------+------------+--------------+------------------+--------------+----------------+------------+-----------+
|187158    |Harbir      |Female        |1924-06-30        |+91 0112009318|Galactosemia    |Rourkela    |H1001      |
|112766    |Brahmdev    |Female        |1948-12-20        |+91 1727749552|Bladder cancer  |Tiruvottiyur|H1016      |
|199252    |Ujjawal     |Male          |1980-04-16        |+91 8547451606|Kidney cancer   |Berhampur   |H1009      |
|133424    |Ballari     |Female        |1969-09-25        |+91 0106026841|Suicide         |Bihar Sharif|H1017      |
|172579    |Devnath     |Female        |1946-05-01        |+91 1868774631|Food allergy    |Bidhannagar |H1019      |
|171320    |Atasi       |Male          |1967-10-02        |+91 9

In [0]:
#patient_data.printSchema()

root
 |-- Patient_id: integer (nullable = true)
 |-- Patient_name: string (nullable = true)
 |-- patient_gender: string (nullable = true)
 |-- patient_birth_date: date (nullable = true)
 |-- patient_phone: string (nullable = true)
 |-- disease_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- hospital_id: string (nullable = true)



In [0]:
patent_trans_data = patient_data.select("Patient_id","Patient_name","patient_gender","patient_birth_date","disease_name","city","hospital_id").distinct()

In [0]:
#patient_data.select([count(when(col(c).isNull(), c)).alias(c) for c in patient_data.columns]).show()

+----------+------------+--------------+------------------+-------------+------------+----+-----------+
|Patient_id|Patient_name|patient_gender|patient_birth_date|patient_phone|disease_name|city|hospital_id|
+----------+------------+--------------+------------------+-------------+------------+----+-----------+
|         0|          17|             0|                 0|            2|           0|   0|          0|
+----------+------------+--------------+------------------+-------------+------------+----+-----------+



In [0]:
#patient_data.groupby(patient_data.columns).count().where("count > 1").show() 

+----------+------------+--------------+------------------+-------------+------------+----+-----------+-----+
|Patient_id|Patient_name|patient_gender|patient_birth_date|patient_phone|disease_name|city|hospital_id|count|
+----------+------------+--------------+------------------+-------------+------------+----+-----------+-----+
+----------+------------+--------------+------------------+-------------+------------+----+-----------+-----+



In [0]:
patient_data = patient_data.dropDuplicates()

In [0]:
#patent_trans_data.show(5)

+----------+------------+--------------+------------------+-------------------+--------------+-----------+
|Patient_id|Patient_name|patient_gender|patient_birth_date|       disease_name|          city|hospital_id|
+----------+------------+--------------+------------------+-------------------+--------------+-----------+
|    197503|        NULL|        Female|        1968-07-02|             Stroke|       Gwalior|      H1009|
|    162665|    Kanhaiya|        Female|        1954-10-19|Alcohol consumption|        Mysore|      H1009|
|    122592|  Vaijayanti|          Male|        1920-11-13|            Cholera|Mira-Bhayandar|      H1009|
|    150189|     Dheeman|          Male|        1945-05-04|            Measles|        Mysore|      H1017|
|    117945|        NULL|          Male|        1955-12-24|           Glaucoma|    Karimnagar|      H1009|
+----------+------------+--------------+------------------+-------------------+--------------+-----------+
only showing top 5 rows



In [0]:
patent_trans_data = patent_trans_data.fillna({"Patient_name":"Guest/NA"})

In [0]:
#patent_trans_data.show(5,False)

+----------+------------+--------------+------------------+-------------------+--------------+-----------+
|Patient_id|Patient_name|patient_gender|patient_birth_date|disease_name       |city          |hospital_id|
+----------+------------+--------------+------------------+-------------------+--------------+-----------+
|197503    |Guest/NA    |Female        |1968-07-02        |Stroke             |Gwalior       |H1009      |
|162665    |Kanhaiya    |Female        |1954-10-19        |Alcohol consumption|Mysore        |H1009      |
|122592    |Vaijayanti  |Male          |1920-11-13        |Cholera            |Mira-Bhayandar|H1009      |
|150189    |Dheeman     |Male          |1945-05-04        |Measles            |Mysore        |H1017      |
|117945    |Guest/NA    |Male          |1955-12-24        |Glaucoma           |Karimnagar    |H1009      |
+----------+------------+--------------+------------------+-------------------+--------------+-----------+
only showing top 5 rows



In [0]:
patent_trans_data = patent_trans_data.withColumn("Patient_age", (months_between(current_date(), col('patient_birth_date'))/12).cast("int"))

In [0]:
#atent_trans_data.show(6,False)

+----------+------------+--------------+------------------+-------------------+--------------+-----------+-----------+
|Patient_id|Patient_name|patient_gender|patient_birth_date|disease_name       |city          |hospital_id|Patient_age|
+----------+------------+--------------+------------------+-------------------+--------------+-----------+-----------+
|197503    |Guest/NA    |Female        |1968-07-02        |Stroke             |Gwalior       |H1009      |55         |
|162665    |Kanhaiya    |Female        |1954-10-19        |Alcohol consumption|Mysore        |H1009      |69         |
|122592    |Vaijayanti  |Male          |1920-11-13        |Cholera            |Mira-Bhayandar|H1009      |103        |
|150189    |Dheeman     |Male          |1945-05-04        |Measles            |Mysore        |H1017      |78         |
|117945    |Guest/NA    |Male          |1955-12-24        |Glaucoma           |Karimnagar    |H1009      |67         |
|130339    |Aakar       |Female        |1925-03-

In [0]:
patent_trans_data = patent_trans_data.drop(col("patient_birth_date"))

In [0]:
#patent_trans_data.show(5,False)

+----------+------------+--------------+-------------------+--------------+-----------+-----------+
|Patient_id|Patient_name|patient_gender|disease_name       |city          |hospital_id|Patient_age|
+----------+------------+--------------+-------------------+--------------+-----------+-----------+
|197503    |Guest/NA    |Female        |Stroke             |Gwalior       |H1009      |55         |
|162665    |Kanhaiya    |Female        |Alcohol consumption|Mysore        |H1009      |69         |
|122592    |Vaijayanti  |Male          |Cholera            |Mira-Bhayandar|H1009      |103        |
|150189    |Dheeman     |Male          |Measles            |Mysore        |H1017      |78         |
|117945    |Guest/NA    |Male          |Glaucoma           |Karimnagar    |H1009      |67         |
+----------+------------+--------------+-------------------+--------------+-----------+-----------+
only showing top 5 rows



In [0]:
#patent_trans_data.select("*").filter(col("patient_id")==121783).show(5)

+----------+------------+--------------+--------------+--------+-----------+-----------+
|Patient_id|Patient_name|patient_gender|  disease_name|    city|hospital_id|Patient_age|
+----------+------------+--------------+--------------+--------+-----------+-----------+
|    121783|     Paridhi|        Female|Bladder cancer|Jabalpur|      H1013|         64|
+----------+------------+--------------+--------------+--------+-----------+-----------+



In [0]:
output_container_path = "abfss://stagginghealthcaredata@carehealthcareblobsa.dfs.core.windows.net"
output_blob_folder = "stagingdatasets/" 
patent_trans_data.coalesce(1).write.mode("overwrite").option("header", "true").format("com.databricks.spark.csv").save(output_blob_folder)
files = dbutils.fs.ls(output_blob_folder)
output_file = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(output_file[0].path, "%s/patientdatastagging.csv" % output_container_path)

True